# Training Results Analysis

Parse and visualize training results from SLURM log files.


In [ ]:
# Setup
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Add project root to path
project_root = Path().resolve().parent.parent
sys.path.insert(0, str(project_root))

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11


In [ ]:
# Import required modules
import re
from typing import Dict

# Define functions
def parse_log_file(log_path: Path) -> Dict:
    """Parse SLURM log file and extract key metrics."""
    results = {
        'job_id': None, 'node': None, 'start_time': None, 'end_time': None, 'duration': None,
        'train_samples': None, 'val_samples': None, 'test_samples': None,
        'train_class_dist': {}, 'val_class_dist': {}, 'test_class_dist': {},
        'best_val_loss': None, 'test_loss': None, 'test_accuracy': None,
        'test_icu_stays': None, 'epochs': [],
    }
    
    if not log_path.exists():
        print(f"❌ Log-Datei nicht gefunden: {log_path}")
        return results
    
    content = log_path.read_text()
    
    job_match = re.search(r'Job ID: (\d+)', content)
    if job_match:
        results['job_id'] = job_match.group(1)
    
    node_match = re.search(r'Node: (\S+)', content)
    if node_match:
        results['node'] = node_match.group(1)
    
    start_match = re.search(r'Start time: (.+)', content)
    if start_match:
        results['start_time'] = start_match.group(1)
    
    end_match = re.search(r'End time: (.+)', content)
    if end_match:
        results['end_time'] = end_match.group(1)
    
    duration_match = re.search(r'Job Wall-clock time: (.+)', content)
    if duration_match:
        results['duration'] = duration_match.group(1)
    
    train_match = re.search(r'ECGDataset Statistics:.*?Total samples: (\d+)', content, re.DOTALL)
    if train_match:
        results['train_samples'] = int(train_match.group(1))
    
    class_dist_matches = re.findall(r'Class distribution \(after filtering\): (\{[^}]+\})', content)
    if len(class_dist_matches) >= 1:
        results['train_class_dist'] = eval(class_dist_matches[0])
    if len(class_dist_matches) >= 2:
        results['val_class_dist'] = eval(class_dist_matches[1])
    if len(class_dist_matches) >= 3:
        results['test_class_dist'] = eval(class_dist_matches[2])
    
    if results['train_class_dist']:
        results['train_samples'] = sum(results['train_class_dist'].values())
    if results['val_class_dist']:
        results['val_samples'] = sum(results['val_class_dist'].values())
    if results['test_class_dist']:
        results['test_samples'] = sum(results['test_class_dist'].values())
    
    val_loss_match = re.search(r'Best validation loss: ([\d.]+)', content)
    if val_loss_match:
        results['best_val_loss'] = float(val_loss_match.group(1))
    
    test_loss_match = re.search(r'Test Loss: ([\d.]+)', content)
    if test_loss_match:
        results['test_loss'] = float(test_loss_match.group(1))
    
    test_acc_match = re.search(r'Test Accuracy:\s+([\d.]+)', content)
    if test_acc_match:
        results['test_accuracy'] = float(test_acc_match.group(1))
    
    test_balanced_acc_match = re.search(r'Balanced Accuracy:\s*([\d.]+)', content)
    if test_balanced_acc_match:
        results['test_balanced_accuracy'] = float(test_balanced_acc_match.group(1))
    
    test_macro_precision_match = re.search(r'Macro Precision:\s*([\d.]+)', content)
    if test_macro_precision_match:
        results['test_macro_precision'] = float(test_macro_precision_match.group(1))
    
    test_macro_recall_match = re.search(r'Macro Recall:\s*([\d.]+)', content)
    if test_macro_recall_match:
        results['test_macro_recall'] = float(test_macro_recall_match.group(1))
    
    test_macro_f1_match = re.search(r'Macro F1-Score:\s*([\d.]+)', content)
    if test_macro_f1_match:
        results['test_macro_f1'] = float(test_macro_f1_match.group(1))
    
    per_class_section = re.search(r'Per-Class Metrics:(.*?)(?:\n\s*\n|Confusion Matrix:)', content, re.DOTALL)
    if per_class_section:
        per_class_lines = per_class_section.group(1).strip().split('\n')
        results['per_class_metrics'] = {}
        for line in per_class_lines[2:]:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) >= 4 and parts[0].isdigit():
                try:
                    cls = int(parts[0])
                    results['per_class_metrics'][cls] = {
                        'precision': float(parts[1]),
                        'recall': float(parts[2]),
                        'f1': float(parts[3])
                    }
                except (ValueError, IndexError):
                    continue
    
    cm_section = re.search(r'Confusion Matrix:\s*\n(.*?)(?:\n\s*\n|End time|Job completed)', content, re.DOTALL)
    if cm_section:
        cm_lines = [line.strip() for line in cm_section.group(1).strip().split('\n') if line.strip()]
        if cm_lines:
            results['confusion_matrix'] = []
            for line in cm_lines:
                line = line.strip()
                if not line:
                    continue
                if line and line[0].isdigit():
                    parts = line.split()
                    if len(parts) > 1:
                        try:
                            row = [int(x) for x in parts[1:]]
                            results['confusion_matrix'].append(row)
                        except (ValueError, IndexError):
                            continue
    
    test_stays_match = re.search(r'Number of ICU stays evaluated: (\d+)', content)
    if test_stays_match:
        results['test_icu_stays'] = int(test_stays_match.group(1))
    
    epoch_matches = re.findall(r'Epoch (\d+)/(\d+).*?Train Loss: ([\d.]+).*?Val Loss: ([\d.]+)', content)
    if epoch_matches:
        results['epochs'] = [
            {
                'epoch': int(e[0]),
                'total': int(e[1]),
                'train_loss': float(e[2]),
                'val_loss': float(e[3])
            }
            for e in epoch_matches[-5:]
        ]
    
    return results

def format_class_distribution(dist: Dict[int, int]) -> str:
    """Format class distribution as a compact string."""
    if not dist:
        return "N/A"
    total = sum(dist.values())
    percentages = {k: (v / total * 100) for k, v in dist.items()}
    return f"Total: {total} | " + " | ".join([f"C{k}: {v} ({p:.1f}%)" for k, v, p in 
                                               sorted([(k, v, percentages[k]) for k, v in dist.items()])])

def print_results(results: Dict):
    """Print results in a formatted way."""
    print("=" * 80)
    print("📊 TRAINING RESULTS SUMMARY")
    print("=" * 80)
    
    print("\n🔹 Job Information:")
    print(f"   Job ID:      {results['job_id'] or 'N/A'}")
    print(f"   Node:        {results['node'] or 'N/A'}")
    print(f"   Start:       {results['start_time'] or 'N/A'}")
    print(f"   End:         {results['end_time'] or 'N/A'}")
    print(f"   Duration:    {results['duration'] or 'N/A'}")
    
    print("\n🔹 Dataset Split:")
    print(f"   Train:       {results['train_samples'] or 'N/A':,} samples")
    print(f"   Validation:  {results['val_samples'] or 'N/A':,} samples")
    print(f"   Test:        {results['test_samples'] or 'N/A':,} samples")
    total = (results['train_samples'] or 0) + (results['val_samples'] or 0) + (results['test_samples'] or 0)
    print(f"   Total:       {total:,} samples")
    
    print("\n🔹 Class Distribution:")
    print(f"   Train:       {format_class_distribution(results['train_class_dist'])}")
    print(f"   Validation:  {format_class_distribution(results['val_class_dist'])}")
    print(f"   Test:        {format_class_distribution(results['test_class_dist'])}")
    
    print("\n🔹 Model Performance:")
    if results['best_val_loss'] is not None:
        print(f"   Best Validation Loss: {results['best_val_loss']:.4f}")
    if results['test_loss'] is not None:
        print(f"   Test Loss:            {results['test_loss']:.4f}")
    if results['test_accuracy'] is not None:
        print(f"   Test Accuracy:        {results['test_accuracy']:.4f} ({results['test_accuracy']*100:.2f}%)")
    if results.get('test_balanced_accuracy') is not None:
        print(f"   Balanced Accuracy:    {results['test_balanced_accuracy']:.4f} ({results['test_balanced_accuracy']*100:.2f}%)")
    if results.get('test_macro_precision') is not None:
        print(f"   Macro Precision:      {results['test_macro_precision']:.4f}")
    if results.get('test_macro_recall') is not None:
        print(f"   Macro Recall:         {results['test_macro_recall']:.4f}")
    if results.get('test_macro_f1') is not None:
        print(f"   Macro F1-Score:       {results['test_macro_f1']:.4f}")
    if results['test_icu_stays'] is not None:
        print(f"   Test ICU Stays:       {results['test_icu_stays']:,}")
    
    if results.get('per_class_metrics'):
        print("\n🔹 Per-Class Metrics:")
        print(f"   {'Class':<8} {'Precision':<12} {'Recall':<12} {'F1-Score':<12}")
        print("   " + "-" * 44)
        for cls in sorted(results['per_class_metrics'].keys()):
            metrics = results['per_class_metrics'][cls]
            print(f"   {cls:<8} {metrics['precision']:<12.4f} {metrics['recall']:<12.4f} {metrics['f1']:<12.4f}")
    
    if results.get('confusion_matrix'):
        print("\n🔹 Confusion Matrix:")
        cm = results['confusion_matrix']
        if cm and len(cm) > 0:
            num_rows = len(cm)
            num_cols = len(cm[0]) if cm[0] else 0
            num_classes = max(num_rows, num_cols)
            print("   " + " ".join([f"{i:>6}" for i in range(num_classes)]))
            for i in range(num_classes):
                if i < num_rows and len(cm[i]) >= num_classes:
                    row_str = f"   {i} " + " ".join([f"{cm[i][j]:>6}" for j in range(num_classes)])
                    print(row_str)
                elif i < num_rows:
                    row = cm[i] + [0] * (num_classes - len(cm[i]))
                    row_str = f"   {i} " + " ".join([f"{row[j]:>6}" for j in range(num_classes)])
                    print(row_str)
                else:
                    row_str = f"   {i} " + " ".join(["     0"] * num_classes)
                    print(row_str)
    
    if results['epochs']:
        print("\n🔹 Last 5 Epochs:")
        print(f"   {'Epoch':<8} {'Train Loss':<12} {'Val Loss':<12}")
        print("   " + "-" * 32)
        for ep in results['epochs']:
            print(f"   {ep['epoch']:<8} {ep['train_loss']:<12.4f} {ep['val_loss']:<12.4f}")
    
    print("\n" + "=" * 80)
    if results.get('job_id'):
        checkpoint_path = f"outputs/checkpoints/CNNScratch_best_{results['job_id']}.pt"
        print(f"✅ Checkpoints: {checkpoint_path}")
    else:
        print("✅ Checkpoints: outputs/checkpoints/CNNScratch_best_<JOB_ID>.pt (Job ID not found)")
    print("=" * 80)


In [ ]:
# Specify job ID or use latest
job_id = None  # Set to specific job ID, or leave None to use latest

if job_id is None:
    # Find latest log file
    log_dir = Path("outputs/logs")
    if log_dir.exists():
        log_files = sorted(log_dir.glob("slurm_*.out"), key=lambda p: p.stat().st_mtime, reverse=True)
        if log_files:
            job_id = log_files[0].stem.replace("slurm_", "")
            print(f"Using latest log file: {log_files[0].name}")
        else:
            print("No log files found!")
    else:
        print("Log directory not found!")
else:
    print(f"Using job ID: {job_id}")


In [ ]:
# Parse and display results
if job_id:
    log_path = Path(f"outputs/logs/slurm_{job_id}.out")
    
    if log_path.exists():
        results = parse_log_file(log_path)
        print_results(results)
        
        # Visualize training history if available
        if results.get('epochs'):
            epochs_data = results['epochs']
            epochs = [e['epoch'] for e in epochs_data]
            train_losses = [e['train_loss'] for e in epochs_data]
            val_losses = [e['val_loss'] for e in epochs_data]
            
            fig, ax = plt.subplots(figsize=(12, 6))
            ax.plot(epochs, train_losses, label='Train Loss', marker='o')
            ax.plot(epochs, val_losses, label='Val Loss', marker='s')
            ax.set_xlabel('Epoch', fontsize=12)
            ax.set_ylabel('Loss', fontsize=12)
            ax.set_title('Training History', fontsize=14, fontweight='bold')
            ax.legend()
            ax.grid(alpha=0.3)
            plt.tight_layout()
            plt.show()
    else:
        print(f"Log file not found: {log_path}")
